In [8]:
import pennylane as qml
import numpy as np

In [9]:
# Create 4 qubit QFT
dev = qml.device("default.qubit", wires=4)

@qml.qnode(dev)
def qft():
    
    qml.Hadamard(3)
    #Qubit 3 rotations
    qml.ControlledPhaseShift(np.pi/2, wires=[2,3])
    qml.ControlledPhaseShift(np.pi/4, wires=[1,3])
    qml.ControlledPhaseShift(np.pi/8, wires=[0,3])

    qml.Hadamard(2)
    #Qubit 2 rotations
    qml.ControlledPhaseShift(np.pi/2, wires=[1,2])
    qml.ControlledPhaseShift(np.pi/4, wires=[0,2])
    
    qml.Hadamard(1)
    #Qubit 1 rotations
    qml.ControlledPhaseShift(np.pi/2, wires=[0,1])
    
    qml.Hadamard(0)
    
    #Swap gates
    qml.SWAP(wires=[0,3])
    qml.SWAP(wires=[1,2])

    return qml.probs(wires=[0, 1, 2, 3])
    #return qml.state()
    
qft()

array([0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625,
       0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625])

In [10]:
@qml.qnode(dev)
def iqft():
    #same as QFT but reverse order with negative phases
    # First apply SWAP gates
    qml.SWAP(wires=[0,3])
    qml.SWAP(wires=[1,2])
    
    # Qubit 0 operations
    qml.Hadamard(0)
    
    # Qubit 1 operations (negative phase shifts)
    qml.ControlledPhaseShift(-np.pi/2, wires=[0,1])
    qml.Hadamard(1)
    
    # Qubit 2 operations
    qml.ControlledPhaseShift(-np.pi/4, wires=[0,2])
    qml.ControlledPhaseShift(-np.pi/2, wires=[1,2])
    qml.Hadamard(2)
    
    # Qubit 3 operations
    qml.ControlledPhaseShift(-np.pi/8, wires=[0,3])
    qml.ControlledPhaseShift(-np.pi/4, wires=[1,3])
    qml.ControlledPhaseShift(-np.pi/2, wires=[2,3])
    qml.Hadamard(3)
    
    return qml.probs(wires=[0, 1, 2, 3])
    # return qml.state()

iqft()

array([0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625,
       0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625])

In [11]:
@qml.qnode(dev)
#Circuit to get state after qft
def qftCircuit(initialState):
    qml.QubitStateVector(initialState, wires=[0,1,2,3])
    qft()
    return qml.state()

#Circuit to get state after qft and iqft
@qml.qnode(dev)
def iqftCircuit(initialState):
    qml.QubitStateVector(initialState, wires=[0,1,2,3])
    qft()
    iqft()
    return qml.state()

# make random 4 qubit state
np.random.seed(12)
randomState = np.random.rand(16) + 1j*np.random.rand(16)
randomState = randomState / np.linalg.norm(randomState)

qftState = qftCircuit(randomState)
finalState = iqftCircuit(randomState)

print("=== Initial State ===")
print(np.array2string(randomState, precision=4, suppress_small=True))
print("\n=== After QFT ===")
print(np.array2string(qftState, precision=4, suppress_small=True))
print("\n=== After QFT + IQFT ===")
print(np.array2string(finalState, precision=4, suppress_small=True))

=== Initial State ===
[0.0481+0.1721j 0.2308+0.1513j 0.0821+0.2395j 0.1664+0.0501j
 0.0045+0.2384j 0.2865+0.0065j 0.2808+0.0422j 0.0104+0.0363j
 0.2984+0.0966j 0.0428+0.2094j 0.0885+0.1469j 0.189 +0.2545j
 0.2944+0.0903j 0.2659+0.2286j 0.0007+0.2191j 0.1625+0.1021j]

=== After QFT ===
[0.0481+0.1721j 0.2308+0.1513j 0.0821+0.2395j 0.1664+0.0501j
 0.0045+0.2384j 0.2865+0.0065j 0.2808+0.0422j 0.0104+0.0363j
 0.2984+0.0966j 0.0428+0.2094j 0.0885+0.1469j 0.189 +0.2545j
 0.2944+0.0903j 0.2659+0.2286j 0.0007+0.2191j 0.1625+0.1021j]

=== After QFT + IQFT ===
[0.0481+0.1721j 0.2308+0.1513j 0.0821+0.2395j 0.1664+0.0501j
 0.0045+0.2384j 0.2865+0.0065j 0.2808+0.0422j 0.0104+0.0363j
 0.2984+0.0966j 0.0428+0.2094j 0.0885+0.1469j 0.189 +0.2545j
 0.2944+0.0903j 0.2659+0.2286j 0.0007+0.2191j 0.1625+0.1021j]
